In [1]:
import tensorflow as tf
import numpy as np


In [2]:
import io,os,sys,types
from IPython import get_ipython #for import notebook
from nbformat import read #for import notebook
from IPython.core.interactiveshell import InteractiveShell #for import notebook

In [4]:
def find_notebook(fullname, path=None): #for import notebook
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path

In [5]:
class NotebookLoader(object): #for import notebook
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod
class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

In [7]:
sys.meta_path.append(NotebookFinder())
import myprepdata

In [2]:

class attention_decoder(object):
    def __init__(self, layers, directions, hiddensize, input_length, 
                 output_v, output_length, teacher_forcing): # encoder_out, state will be feed
        
        layers = 1
        directions = 1
        hiddensize = 100
        self.hiddensize = hiddensize
        
        with tf.variable_scope('input') as scope:
            input_length = 484
            self.input_v = tf.placeholder(dtype='float32', shape=[1, input_length, self.hiddensize])#encoder_output
            
            
        with tf.variable_scope('output') as scope:
            self.output = tf.Variable(tf.zeros([self.hiddensize]),dtype='float32') #my output segment
            self.output_v = output_v #real full output
 

        with tf.variable_scope('grustate') as scope:
            self.grustate = tf.placeholder(dtype='float32', shape=[1, self.hiddensize])#encoder_state
            
        teacher_forcing = False
        
        
        self.gru = tf.nn.rnn_cell.GRUCell(num_units = hiddensize)
        
        with tf.variable_scope('var') as scope:
            self.W1 = tf.Variable(tf.zeros([self.hiddensize*2, input_length]), dtype='float32')
            self.W2 = tf.Variable(tf.zeros([self.hiddensize*2, self.hiddensize]), dtype='float32')
            self.b1 = tf.Variable(tf.zeros([input_length]), dtype='float32')
            self.b2 = tf.Variable(tf.zeros([self.hiddensize]), dtype='float32')
            
        encoder_output_state = tf.concat([self.output, self.grustate],1) # 1*100 + 1*100 = 1*200
        
        length = np.zeros(output_length)
        self.last_status = tf.scan(self.nt_atten, length , initializer = encoder_output_state) #my full output
        
    def nt_atten(self, grustate_output ,ignore): #not teacher forcing

        with tf.variable_scope("split") as scope:
            self.grustate, self.output = tf.split(grustate_output, num_or_size_splits=2, axis=1)
            _ = ignore
        with tf.variable_scope("gru") as scope:

            self.grustate, self.output =self.gru.call(self.grustate, self.output )

            #word = myprepdata.findwordsfromvec(output.eval())
            #print(word)
            #np_output = myprepdata.findvecfromwords(word)
            #output = tf.Variable(np_output, dtype="float32")

        with tf.variable_scope("atten") as scope:
            attn = tf.concat([self.grustate, self.output],1)#concat and linear
            attn = tf.add(tf.matmul(attn , self.W1),self.b1) # needs W initialize
            attn = tf.nn.softmax(tf.nn.relu(attn))

        with tf.variable_scope("bmm") as scope:
            bmm = tf.matmul( attn,self.input_v)

        with tf.variable_scope("comb") as scope:
            attn_com = tf.concat([bmm, self.output],1 )
            attn_com = tf.add(tf.matmul(attn_com,self.W2),self.b2)
            self.output = tf.nn.relu(attn_com)

        with tf.variable_scope("concat") as scope:
            grustate_output = tf.concat([self.grustate, self.output],1)

        return grustate_output

    def train(self,sess,input_v,grustate):
        sess.run([self.last_status],feed_dict = {self.input_v:input_v, self.grustate: grustate})

    
    def hidden_init(self,sess):
        return sess.run(tf.global_variables_initializer())


In [ ]:

'''
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter("./testgru")
    writer.add_graph(sess.graph)
    i = 0
    step = 0
    
    print(sess.run([last_status],feed_dict={input_v: np.zeros((484, 100))}))
    
    #s = sess.run(summary)
    #writer.add_summary(s,i)
    i = i + 1
'''

In [8]:
#c = tf.add(c, b)
'''
with tf.variable_scope('gru') as scope:
    a = tf.Variable([[1,0,0,0]],dtype="float32")
    b = tf.Variable([[0,1,0,0]],dtype="float32")
    k = tf.assign(b,tf.subtract(b,a))
    c = tf.Variable([[0,0,1,0]],dtype="float32")
print(a[0][2])
i = 0

with tf.variable_scope('gruu') as scope:
    while(i < 10):
        c = tf.add(b,c)
        c = tf.add(b,c)
        i = i + 1
    #b = tf.add(a,b)
    
with tf.variable_scope('gruuu') as scope:
    ppp = tf.assign(a,tf.subtract(b,a))
    qqq = tf.assign(b,tf.subtract(b,c))
    a_v = tf.summary.histogram("a",a)#not drowed in graph
    b_v = tf.summary.histogram("b",b)
    c_v = tf.summary.histogram("c",c)
'''

Tensor("strided_slice_1:0", shape=(), dtype=float32)


In [ ]:
'''
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter("./testgru")
    writer.add_graph(sess.graph)
    i = 0
    step = 0
    
    print(sess.run([b]))
    print(sess.run([c]))
    print(sess.run([ppp]))
    print(sess.run([b]))
    print(sess.run([ppp]))
    print(sess.run([a]))
    print(sess.run([b]))
    print(sess.run([ppp]))
    print(sess.run([ppp]))

    
    

    s = sess.run(summary)
    writer.add_summary(s,i)
    i = i + 1
'''

In [9]:
'''
a0 = tf.constant([[[1, 1, 1], [2, 2, 2]], [[3, 3, 3], [4, 4, 4]], [[5, 5, 5], [6, 6, 6]]], dtype = 'float32')


b0 = tf.slice(a , [0, 0, 0], [3, 2, 3])
b1 = tf.slice(a , [0, 0, 1], [1, 1, 2])
b2 = tf.slice(a , [0, 0, 0], [1, 1, 3])
b3 = tf.slice(a , [0, 0, 1], [-1, -1, 2])
b4 = tf.slice(a , [0, 0, 0], [1, 0, 2])
b5 = tf.slice(a , [0, 0, 0], [1, 0, 3])
sess = tf.Session()
sess.run(tf.global_variables_initializer())
sess.run([b,b1,b2,b3,b4,b5])
'''

"\na0 = tf.constant([[[1, 1, 1], [2, 2, 2]], [[3, 3, 3], [4, 4, 4]], [[5, 5, 5], [6, 6, 6]]], dtype = 'float32')\n\n\nb0 = tf.slice(a , [0, 0, 0], [3, 2, 3])\nb1 = tf.slice(a , [0, 0, 1], [1, 1, 2])\nb2 = tf.slice(a , [0, 0, 0], [1, 1, 3])\nb3 = tf.slice(a , [0, 0, 1], [-1, -1, 2])\nb4 = tf.slice(a , [0, 0, 0], [1, 0, 2])\nb5 = tf.slice(a , [0, 0, 0], [1, 0, 3])\nsess = tf.Session()\nsess.run(tf.global_variables_initializer())\nsess.run([b,b1,b2,b3,b4,b5])\n"

In [3]:
'''
hiddensize = 100
maxv = 484
teacher_forcing = False
with tf.variable_scope('grustate') as scope:
    grustate = tf.Variable(tf.zeros([1,hiddensize]),dtype='float32')
    
with tf.variable_scope('output') as scope:
    output = tf.Variable(tf.zeros([1,hiddensize]),dtype='float32')
    
with tf.variable_scope('var') as scope:
    W1 = tf.Variable(tf.zeros([hiddensize*2, maxv]), dtype='float32')
    W2 = tf.Variable(tf.zeros([hiddensize*2, hiddensize]), dtype='float32')
    b1 = tf.Variable(tf.zeros([maxv]), dtype='float32')
    b2 = tf.Variable(tf.zeros([hiddensize]), dtype='float32')
    i = 0
    
with tf.variable_scope('input') as scope:
    input_v = tf.placeholder(dtype='float32', shape=[ maxv, hiddensize])

encoder_output_state = tf.concat([output, grustate],1)

def nt_atten(grustate_output ,ignore): #not teacher forcing
    
    with tf.variable_scope("split") as scope:
        grustate, output = tf.split(grustate_output, num_or_size_splits=2, axis=1)
        _ = ignore
    with tf.variable_scope("gru") as scope:
        gru = tf.nn.rnn_cell.GRUCell(num_units = hiddensize)
        grustate, output =gru.call(grustate, output )

        #word = myprepdata.findwordsfromvec(output.eval())
        #print(word)
        #np_output = myprepdata.findvecfromwords(word)
        #output = tf.Variable(np_output, dtype="float32")
        
    with tf.variable_scope("atten") as scope:
        attn = tf.concat([output, grustate],1)#concat and linear
        attn = tf.add(tf.matmul(attn , W1),b1) # needs W initialize
        attn = tf.nn.softmax(tf.nn.relu(attn))
        
    with tf.variable_scope("bmm") as scope:
        bmm = tf.matmul( attn,input_v)
        
    with tf.variable_scope("comb") as scope:
        attn_com = tf.concat([bmm, output],1 )
        attn_com = tf.add(tf.matmul(attn_com,W2),b2)
        output = tf.nn.relu(attn_com)
        
    with tf.variable_scope("concat") as scope:
        grustate_output = tf.concat([output, grustate],1)
    
    return grustate_output

length = np.zeros(5)
last_status = tf.scan(nt_atten, length , initializer = encoder_output_state)
'''
 

In [3]:
'''
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter("./testgru")
    writer.add_graph(sess.graph)
    i = 0
    step = 0
    
    print(sess.run([last_status],feed_dict={input_v: np.zeros((484, 100))}))
    
    #s = sess.run(summary)
    #writer.add_summary(s,i)
    i = i + 1
'''

[array([[[ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       ..., 
       [[ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.]],

       [[ 0.,  0.,  0., ...,  0.,  0.,  0.]]], dtype=float32)]


In [6]:
np.shape(np.array(0))

()

In [5]:
np.shape(np.array([0]))

(1,)